# Sudoku Generation

This notebook is used to create functions and use them to create various sudoku grids using sudoku shuffling techniques.

A valid sudoku table has following properties: (source: https://en.wikipedia.org/wiki/Sudoku_solving_algorithms )
* contains 81 numbers (cells in a 9x9 grid with 9 boxes)
* each box being the intersection of the first, middle, or last 3 rows, and the first, middle, or last 3 columns
* each cell contains number from 1 to 9
* each number can only occur once in each row, column and box

In sudoku, you can do following operations to keep the table valid:
* swap row/column blocks
* swap all occurances of one number with all occurances of another number
* swap rows/columns in a block

By doing such operations, you can create many new sudoku tables that will eventually result in a different problems using one input. 

## Creation of Solved Sudokus

#### Creation of Necessary Functions
Let's now create a set of functions that will randomly apply the rules above on existing sudoku table.

In [92]:
import random 

# swap all occurances of two numbers in a grid
def swap_numbers(grid, x, y):
    return grid.replace(x, 'x').replace(y, x).replace('x', y)

# swap rows within a block
def swap_rows(grid):
    # extract rows
    rows = [grid[i*9:(i+1)*9] for i in range(9)]
    
    # shuffle rows within blocks
    for i in range(0, 9, 3):
        block_rows = rows[i:i+3]
        random.shuffle(block_rows)
        rows[i:i+3] = block_rows
    
    # reconstruct grid
    return ''.join(rows)

# swap columns within a block
def swap_columns(grid):
    # extract columns
    columns = []
    for i in range(9):
        column = [int(grid[j*9 + i]) for j in range(9)]
        columns.append(column)
    
    # shuffle columns within blocks
    for i in range(0, 9, 3):
        block_columns = columns[i:i+3]
        random.shuffle(block_columns)
        columns[i:i+3] = block_columns
    
    # reconstruct grid
    return ''.join(''.join(str(columns[j][i]) for j in range(9)) for i in range(9))

# swap blocks of rows
def swap_row_blocks(grid):
    # extract rows
    rows = [grid[i*9:(i+1)*9] for i in range(9)]
    
    # group rows into blocks
    row_blocks = [rows[i*3:(i+1)*3] for i in range(3)]
    
    # shuffle row blocks
    random.shuffle(row_blocks)
    
    # reconstruct grid
    return ''.join([''.join(block) for block in row_blocks])

# swap blocks of columns
def swap_column_blocks(grid):
    # extract columns
    columns = []
    for i in range(9):
        column = [int(grid[j*9 + i]) for j in range(9)]
        columns.append(column)
    
    # group columns into blocks
    blocks = [columns[i:i+3] for i in range(0, 9, 3)]
    
    # shuffle column blocks
    random.shuffle(blocks)
    
    # reconstruct grid
    shuffled_columns = [col for block in blocks for col in block]
    return ''.join(''.join(str(shuffled_columns[j][i]) for j in range(9)) for i in range(9))

# generate a randomly shuffled sudoku grid
def sudoku_shuffle(grid):
    # swap random times two random numbers
    reps=random.randint(1, 9)
    while reps>0:
        num1, num2 = random.sample('123456789', 2)
        grid = swap_numbers(grid, num1, num2)
        reps-=1
    
    # swap rows within blocks
    grid = swap_row_blocks(grid)
    
    # swap columns within blocks
    grid = swap_column_blocks(grid)
    
    # swap row blocks
    grid = swap_row_blocks(grid)
    
    # swap column blocks
    grid = swap_column_blocks(grid)
    
    return grid

#### Generation of Sudokus

Now let's define 1 sudoku grid that we know is valid. How do we know that? We created a code called `valid_sudoku()` that returns `True` if the specified input follows the rules described at the beginning of this notebook and returns `False` otherwise. The following block of code also contains the definition of `display_sudoku()` function that displays the sudoku in a user friendly format.

After we define our sudoku grid, we apply on it our function `sudoku_shuffle()` to create 1,000 new grids.


In [212]:
def valid_sudoku(grid):
    # check if the input string has exactly 81 characters and contains only digits from 1 to 9
    if len(grid) != 81 or not all(a in '123456789' for a in grid):
        return False
    
    # check rows
    for i in range(9):
        row = [grid[j] for j in range(i*9, (i+1)*9)]
        if not len(row) == len(set(row)):
            return False

    # check columns
    for i in range(9):
        column = [grid[j] for j in range(i, 81, 9)]
        if not len(column) == len(set(column)):
            return False

    # check 3x3 sub-grids (boxes)
    for i in range(0, 81, 27):  # i goes through the first cell of each 3x3 block in a row group
        for j in range(0, 9, 3):  # j goes through each 3x3 block in a row group
            box = [grid[k] for k in (
                i + j, i + j + 1, i + j + 2,
                i + j + 9, i + j + 10, i + j + 11,
                i + j + 18, i + j + 19, i + j + 20
            )]
            if not len(box) == len(set(box)):
                return False

    return True

def display_sudoku(sudoku_str):
    # create a 9x9 grid
    grid = [[int(sudoku_str[i * 9 + j]) for j in range(9)] for i in range(9)]

    # define how to format the grid
    lines = []
    for i, row in enumerate(grid):
        if i % 3 == 0 and i != 0:
            lines.append("-" * 21)  # add horizontal separator

        line = ""
        for j, cell in enumerate(row):
            if j % 3 == 0 and j != 0:
                line += "| "  # add vertical separator
            line += f"{cell} "  # append cell value
        lines.append(line.rstrip())

    # print the grid
    return print("\n".join(lines)) 

Let's now define the sudoku, check if it is valid and display it.

In [213]:
# sudoku definition
sudoku = '123456789456789123789123456234567891567891234891234567345678912678912345912345678'
valid_sudoku(sudoku), display_sudoku(sudoku)

1 2 3 | 4 5 6 | 7 8 9
4 5 6 | 7 8 9 | 1 2 3
7 8 9 | 1 2 3 | 4 5 6
---------------------
2 3 4 | 5 6 7 | 8 9 1
5 6 7 | 8 9 1 | 2 3 4
8 9 1 | 2 3 4 | 5 6 7
---------------------
3 4 5 | 6 7 8 | 9 1 2
6 7 8 | 9 1 2 | 3 4 5
9 1 2 | 3 4 5 | 6 7 8


(True, None)

Finally, we generate 1,000 shuflles of the inital sudoku. While doing so, we check that the created sudokus are properly defined and then we select only unique outputs. We are using a NumPy array for better performance.

In [214]:
import numpy as np
 
# generate 1000 shuffled sudokus
reps_num = 1000
results_array = np.empty((reps_num,), dtype='U81')

for i in range(reps_num):
    if valid_sudoku(sudoku_shuffle(sudoku)):
        results_array[i] = sudoku_shuffle(sudoku)

# select only unique results  
results_array = np.unique(results_array)

# check how many unique sudokus were generated
results_array.shape[0]

995